# Consumo de agua CDMX

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data'
# RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' #####SÓLO TRAE INFO 2016
FILE_NAME = 'aborto_legal'

## Class

In [2]:
from pandas import DataFrame
from typing import Dict

from mariachis.models import BaseClass

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


class AbortoLegal(BaseClass):
    def __init__(self, base_dir: str, file_name: str) -> None: 
        super().__init__(base_dir, file_name)

    def wrangling_aborto(self, df: DataFrame, clean_dict: Dict, date_col: str='fingreso', export_imputed: bool=True, **kwargs):
        # Sólo registros con fecha
        df = df[df[date_col].notnull()].reset_index()
        # Crear variables de fecha
        df = self.date_vars(df, date_col)

        # Obtener las variables numéricas
        multi_num = list(set(clean_dict['vars_num']+clean_dict['vars_numbin']))
        
        # Limpiar si hay texto de variables numéricas
        for col in multi_num:
            df[col] = df[col].map(self.clean_number)

        # Obtener las variables que serán binarias
        bin_cat = list(set(
            clean_dict['vars_yes_no']+
            list(clean_dict['vars_catbin'].keys())+
            list(clean_dict['vars_fill_zero'].keys())+
            list(clean_dict['vars_fill_one'].keys())
        ))

        # Obtener las variables categóricas
        just_cat = list(set(
            clean_dict['vars_just_fill_na']+
            clean_dict['vars_first_word']+
            list(clean_dict['vars_cat'].keys())
        ))
        
        total_cat = just_cat + bin_cat

        # Omitir acentos de variables categóricas
        for col in total_cat:
            df[col] = df[col].map(self.clean_text)

        # Sólo cambiar nulos por "DESCONOCIDO"
        for col in clean_dict['vars_just_fill_na']:
            df[col] = df[col].fillna('DESCONOCIDO')

        # Obtener la primer palabra
        for col in clean_dict['vars_first_word']:
            df[col] = df[col].str.split().str[0]

        # Agrupar categorías
        for col,to_group in clean_dict['vars_cat'].items():
            df[col] = df[col].map(to_group)

        # Agrupar binarias
        for col, to_group in clean_dict['vars_catbin'].items():
            df[col] = df[col].map(to_group)

        # Binarias de sí o no
        for col in clean_dict['vars_yes_no']:
            df[col] = df[col].map({'NO':0, 'SI':1})

        # Lo que quede vacío llenar con 0
        for col, to_fill in clean_dict['vars_fill_zero'].items():
            df[col] = df[col].map(to_fill).fillna(0)

        # Lo que quede vacío llenar con 1
        for col, to_fill in clean_dict['vars_fill_one'].items():
            df[col] = df[col].map(to_fill).fillna(1)

        # Ahora las binarias son también numéricas
        total_num = multi_num+bin_cat

        prep = ColumnTransformer(transformers=[
            ('encode', OneHotEncoder(), just_cat),
            ('impute', IterativeImputer(random_state=22), total_num)
        ])

        # Aplicar para imputar nulos
        default = DataFrame(prep.fit_transform(df[just_cat+total_num]), index=df.index, columns=just_cat+total_num)
        
        # Tal vez el usuario quiere exportar los resultados
        if export_imputed: self.export_csv(df, name_suffix='imputed')

        # Convertir vars_numbin a binarias

        # Estructurar la información limpia
        df = df.pivot_table(**kwargs)
        df.columns = ['_'.join([x for x in col]) if not isinstance(df.columns[0],str) else col for col in df.columns]
        return df

al = AbortoLegal(BASE_DIR, FILE_NAME)
# print(al)

## Data Wrangling

In [3]:
df = al.full_import(api=False)
df.sample()

,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,tanalgesico,cconsejo,panticoncep,fecha_cierre,resultado_ile,medicamento,clues_2,clues_hospital,procile_simplificada,procile_completa
19020,2019.0,AGOSTO,2019-08-14,NO,SOLTERA,20.0,NO ESPECIFICADO,PREPARATORIA,ESTUDIANTE,CATOLICA,...,SI,SI,NaN,NaN,NaN,NaN,NaN,DFSSA003705,MEDICAMENTO,NaN


## Dict

In [4]:
from numpy import nan

# Diccionario para aplicar limpieza por tipo de variable
full_dict = {}

# Sólo cambiar nulos por "DESCONOCIDO"
full_dict['vars_just_fill_na'] = ['entidad', 'motiles', 'desc_servicio', 'anticonceptivo', 'panticoncep']

# Obtener la primer palabra
full_dict['vars_first_word'] = ['anticonceptivo',  'panticoncep']

# Agrupar categorías
full_dict['vars_cat'] = {
    'edocivil_descripcion':{
        'SOLTERA':'SOLTERX', 
        'UNION LIBRE':'UNION_LIBRE', 
        'CASADA':'CASADX',
        'DIVORCIADA':'SEPARADX',
        'SEPARADA':'SEPARADX',
        'VIUDA':'SEPARADX',
        nan:'DESCONOCIDO'
    },
    'ocupacion':{
        'EMPLEADA':'EMPLEADX',
        'ESTUDIANTE':'ESTUDIANTE',
        'TRABAJADORA DEL HOGAR NO REMUNERADA':'TRAB_HOGAR_NO_REMUNERADX',
        'DESEMPLEADA':'OTRO',
        'NO_ESPECIFICADO':'OTRO',
        nan:'OTRO'
    }
}

# Binarias de sí o no
full_dict['vars_yes_no'] = ['autoref', 'consejeria', 'p_consent', 's_complica', 'c_dolor', 'tanalgesico', 'cconsejo', 'resultado_ile', 'medicamento']

# Agrupar binarias
full_dict['vars_catbin'] = {
    'desc_derechohab':{
        'NINGUNO':0,
        'IMSS':1,
        'SEGURO POPULAR':1,
        'OTRA':1,
        'ISSSTE':1
    }
}

# Lo que quede vacío llenar con 0
full_dict['vars_fill_zero'] = {
    'nivel_edu':{
        'LICENCIATURA':1,
        'MAESTRIA':1,
        'DOCTORADO':1
    },
    'parentesco':{'PAREJA':1}
}

# Lo que quede vacío llenar con 1
full_dict['vars_fill_one'] = {
    'religion':{'NINGUNA':0},
    'h_fingreso':{nan:0}
}

# Ahora, limpiar texto de variables numéricas
full_dict['vars_num'] = ['edad', 'menarca', 'fsexual', 'sememb', 'nhijos', 'gesta', 'c_num', 'p_semgest', 'p_diasgesta']
full_dict['vars_numbin'] = ['naborto', 'npartos', 'ncesarea', 'nile']

Siguientes pasos!
- diccionario para reducir variables categóricas
- h_fingreso 0 si está vacía, caso contrario -> 1
- LabelEncoder para categóricas
- KNN Imputer para numéricas
- vars_numbin convertir a binarias
- clustering añomes+municipio con vars_cat+vars_catbin escaladas y calculando ['count','min','mean','max']
- perfilamiento
- agrupar por municipio, cómo es la distribución de los clústeres previos?
- Cómo vender lo que resuelve?

## Transform

In [5]:
from IPython.display import display

X = al.wrangling_aborto(df, full_dict, index=['fingreso_yearmonth', 'entidad', 'alc_o_municipio'], aggfunc=['count','min','median','mean','max'], fill_value=0)

/Users/efraflores/Desktop/hub/dataton_cdmx/venv/lib/python3.9/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning:

[IterativeImputer] Early stopping criterion not reached.



ValueError: Shape of passed values is (77750, 108), indices imply (77750, 34)